# A Brief Collection Analysis and Monte Carlo Projection for Top NFT Collections


## Importing the Libraries

In [2]:
import pandas as pd
import os
# os.getenv
from dotenv import load_dotenv
import hvplot.pandas
import requests

In [2]:
load_dotenv()

rarify_api_key = os.getenv("RARIFY_API_KEY")
display(type(rarify_api_key))

str

# Descriptive Data Analysis

# Part 1

## Fetching from the Rarify API
* We get the data for our nft collections from the rarify API
* We are targeting the collections data endpoint which is the following: "https://api.rarify.tech/data/contracts/{network_id}:{contract_id}/insights/90d"
* We supply the network_id as the blockchain which is Ethereum in our case
* In the first instance we will target the crypto punks collection by supplying it's contract_id to check that our authentication and fetch method works


In [3]:
network_id = "ethereum"
# Crypto Punks
contract_id = "b47e3cd837ddf8e4c57f05d70ab865de6e193bbb"

collections_baseurl = f"https://api.rarify.tech/data/contracts/{network_id}:{contract_id}/insights/90d"

# Use the following code to target a specific token in the collection
token_id = 9620
token_baseurl = f"https://api.rarify.tech/data/tokens/{network_id}:{contract_id}:{token_id}"



"\n    The following function is our base fetch for the collection data using our authorization key stored in the environment\n    variables as well as the url that we supply to the function\n    The url must be supplied with a valid network_id, contract_id, and token_id\n    The function returns the sale_history_data for our targeted collection at the 'history' endpoint\n"

In [5]:
def fetch_rarify_data(url, key):
    """
    The following function is our base fetch for the collection data using our authorization key stored in the environment
    variables as well as the url that we supply to the function
    The url must be supplied with a valid network_id, contract_id, and token_id
    The function returns the sale_history_data for our targeted collection at the 'history' endpoint
    """
    sale_history_data = requests.get(
        url,
        headers={"Authorization": f"Bearer {key}"}
    ).json()
    return sale_history_data['included'][1]['attributes']['history']

## Example Data Object
* We instantiate the punks_return object as a fetch at our api endpoint
* We turn the return into a DataFrame
* We set the 'time' column to a datetime type object
* We set the index of our data to the 'time' column

In [6]:
punks_return = fetch_rarify_data(collections_baseurl, rarify_api_key)
punks_df = pd.DataFrame(punks_return)
punks_df['time'] = pd.to_datetime(punks_df['time'], infer_datetime_format=True)
punks_df = punks_df.set_index('time')

punks_df.head()

,avg_price,max_price,min_price,trades,unique_buyers,volume
time,,,,,,
2022-04-24,70268571000000000000,109950000000000000000,59950000000000000000,7,3,491880000000000000000
2022-04-25,66444761000000000000,124990000000000000000,24240945000000000000,13,10,863781904000000000000
2022-04-26,81953636000000000000,225000000000000000000,61300000000000000000,11,9,901490000000000000000
2022-04-27,76911428000000000000,129990000000000000000,57000000000000000000,7,6,538380000000000000000
2022-04-28,68149166000000000000,90000000000000000000,61000000000000000000,12,11,817790000000000000000


## Type Conversion
* Our numeric data is returned as strings so we must process it
* We use a dict to convert the types of each numeric column to a float type using the df.astype() method

In [7]:
convert_dict = {'avg_price': float,
                'max_price': float,
                'min_price': float,
                'trades': float,
                'unique_buyers': float,
                'volume': float,
               }  
  
punks_df = punks_df.astype(convert_dict)  

## Optional Factoring
* We multiply the numeric data that is given to us in gwei by a factor of 10^-18 to convert it to eth prices

In [8]:
punks_df[['avg_price', 'max_price', 'min_price', 'volume']] = punks_df[['avg_price', 'max_price', 'min_price', 'volume']] * 10**-18

In [9]:
punks_df.head()

,avg_price,max_price,min_price,trades,unique_buyers,volume
time,,,,,,
2022-04-24,70.268571,109.95,59.950000,7.0,3.0,491.880000
2022-04-25,66.444761,124.99,24.240945,13.0,10.0,863.781904
2022-04-26,81.953636,225.00,61.300000,11.0,9.0,901.490000
2022-04-27,76.911428,129.99,57.000000,7.0,6.0,538.380000
2022-04-28,68.149166,90.00,61.000000,12.0,11.0,817.790000


## Preliminary Analysis


### Terms
* Collateral Discount Factor: A percentage that the collateral's value must be discounted in order to ensure a safe return for the lender should the borrower default on his loan. This value differs by the type of asset and is somewhat arbitrary but is based largely on expert appraisal (ie. a car used as collateral may grant the borrower a loan of 50% of the appraised value of the car. The collateral discount factor would be 50%). 

* Collateral Coverage Ratio (CCR): The discounted value of the collateralized asset over the value of a loan that a borrower is looking to receive. A higher CCR (over 1.0) indicates sufficient collateral which will cover the value of the loan at the discounted value of the collateral. (ie. John would like a loan of 10,000 and puts his car, worth 25,000 up as collateral. If a 50% collateral discount factor is applied to John's car, the resulting CCR is 1.225. This would be a safe loan for the lender because he could easily cover his costs, and profit, should the borrower default).


In [10]:
# Standard deviation for the minimum price of the Punks collection
punks_df['min_price'].std()

21.32348263924302

In [11]:
# Display the minimum, average and maximum price for the collection along time
punks_df[['min_price', 'avg_price', 'max_price']].hvplot()

:NdOverlay   [Variable]
   :Curve   [time]   (value)

In [12]:
# This plot is hard to read so we will just plot the average price along with the mean of the average price
punks_df['mean_avg'] = punks_df['avg_price'].mean()
punks_df[['avg_price', 'mean_avg']].hvplot()

:NdOverlay   [Variable]
   :Curve   [time]   (value)

In [13]:
# We take a look at the min_price
punks_df["mean_min"] = punks_df['min_price'].mean()
punks_df[['mean_min', 'min_price']].hvplot()


:NdOverlay   [Variable]
   :Curve   [time]   (value)

In [14]:
punks_df['min_price'].rolling(window=10).std().hvplot(title="min_price standard deviation rolling window=10 days")

:Curve   [time]   (min_price)

Based on the plot it looks like there is little if any trend in the data from the start to the end of the previous 90 days. This may actually be a good signal as it shows that items from the collection may provide stable collateral. For the stability and value of this collection, we would apply a relatively low collateral discount factor for this asset based on its performance and its projected performance overtime. However, the standard deviation of the asset is quite high, this is largely due to the illiquity of NFTs and the relatively few sales that occur on a given day. NFTs in general should be granted a relatively high collateral discount factor compared to other asset classses.

# Part 2

# Analyzing a Series of Collections
* First we aggregate a series of reputable collections from opensea and their contract addresses
* I selected the following collections, but any number of collections would work for analysis:
* *Bored Ape Yacht Club*, *Crypto Punks*, *Clone X*, *Doodles*, *NeoTokyo*, and *Mfers*
* These are all some of the highest performers on OpenSea

In [15]:
# list of collection addresses: 
# bape: 0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D
# punks: b47e3cd837ddf8e4c57f05d70ab865de6e193bbb
# clone x: 0x49cF6f5d44E70224e2E23fDcdd2C053F30aDA28B
# doodles: 0x8a90CAb2b38dba80c64b7734e58Ee1dB38B8992e
# neotokyo: 0xb668beB1Fa440F6cF2Da0399f8C28caB993Bdd65
# mfers: 0x79FCDEF22feeD20eDDacbB2587640e45491b757f

def get_collections_data(contract_ids: dict, rarify_api_key: str):
    """
    *The following function is quite messy and I will clean it up at a later time but it will work for now.*
    This function aggregates the data from a selection of NFT collections into a double-layered DataFrame which can be used to run a Monte Carlo simulation
    
    :param contract_ids: (type: dict) Houses the contract addresses and the collection names
    :param rarify_api_key: (type: str) Your authentication key from the rarify API
    
    The function iterates through the dictionary of addresses that you supply to it and makes an API call for each address.
    It then takes the relevant data and turns it into a DataFrame object.
    We then preprocess the data like we did before, formatting and setting the index as the 'time' column,
    and converting the string numbers to integers using the df.astype() method. We also convert the prices to eth from gwei using a factor. 
    We then append the most recently constructed dataframe to the list that we instantiated at the top of the function
    
    :returns: A concatenation of all the DataFrames that are present in the DataFrame list that we constructed.


    *There is obviously much more elegant way to conduct this process so let me know if you have a cleaner way of doing this*

    """
    df_list = []
    network_id = "ethereum"
    convert_dict = {
                    'avg_price': float,
                    'max_price': float,
                    'min_price': float,
                    'trades': float,
                    'unique_buyers': float,
                    'volume': float,
                   }  
    for address in contract_ids.values():
        contract_id = address
        collections_baseurl = f"https://api.rarify.tech/data/contracts/{network_id}:{contract_id}/insights/90d"
        curr_df = pd.DataFrame(fetch_rarify_data(collections_baseurl, rarify_api_key))
        curr_df['time'] = pd.to_datetime(curr_df['time'], infer_datetime_format=True)
        curr_df = curr_df.set_index('time')
        curr_df = curr_df.astype(convert_dict)
        curr_df[['avg_price', 'max_price', 'min_price', 'volume']] = curr_df[['avg_price', 'max_price', 'min_price', 'volume']] * 10**-18
        df_list.append(curr_df)
    sum_df = pd.concat(df_list, axis=1, keys=contract_ids.keys())
    return sum_df



In [16]:
# I might use these as functions inside the main function at some point but I will have to restructure the framework
# So for now I will set these functions aside here
def set_time_index(df):
    df['time'] = pd.to_datetime(df['time'], infer_datetime_format=True)
    df = df.set_index('time')
    return df

def convert_str_int(df):
    convert_dict = {'avg_price': float,
                'max_price': float,
                'min_price': float,
                'trades': float,
                'unique_buyers': float,
                'volume': float,
               }  
    df = df.astype(convert_dict) 
    return df

## For Some Reason when running this block you will get a key-value error the first couple of times running the function. If you keep running it eventually it will fetch. I can't figure this issue out but my suspicion is that it is an issue with the kernel.

## Please let me know if you know what the bug is

In [17]:
# The collections that we will take a look at with their contract addresses
contract_ids = {
                "bape": "0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D", 
                "punks": "b47e3cd837ddf8e4c57f05d70ab865de6e193bbb", 
                "clonex": "0x49cF6f5d44E70224e2E23fDcdd2C053F30aDA28B",
                "doodles": "0x8a90CAb2b38dba80c64b7734e58Ee1dB38B8992e",
                "neotokyo": "0xb668beB1Fa440F6cF2Da0399f8C28caB993Bdd65",
                "mfers": "0x79FCDEF22feeD20eDDacbB2587640e45491b757f",
}

# Store the resulting concatenated DataFrame in a sum_df object

sum_df = get_collections_data(contract_ids, rarify_api_key)

In [18]:
sum_df.head()

bape                                                         \
             avg_price max_price  min_price trades unique_buyers       volume   
time                                                                            
2022-04-24  106.998594   150.000  14.720000   27.0          13.0  2888.962050   
2022-04-25   95.408970   187.005   0.005000   98.0          37.0  9350.079083   
2022-04-26   99.093631   186.000   0.005000   39.0          25.0  3864.651618   
2022-04-27   93.857465   190.000   1.073615   43.0          19.0  4035.871030   
2022-04-28  130.485907   462.000   3.005000   59.0          31.0  7698.668550   

                punks                              ...  neotokyo         \
            avg_price max_price  min_price trades  ... min_price trades   
time                                               ...                    
2022-04-24  70.268571    109.95  59.950000    7.0  ...       NaN    NaN   
2022-04-25  66.444761    124.99  24.240945   13.0  ...      8.70    4.0   
2022-04-26  81.953636    225.00  61.300000   11.0  ...      7.00    3.0   
2022-04-27  76.911428    129.99  57.000000    7.0  ...      6.99    4.0   
2022-04-28  68.149166     90.00  61.000000   12.0  ...      7.90    2.0   

                                    mfers                             \
           unique_buyers volume avg_price max_price min_price trades   
time                                                                   
2022-04-24           NaN    NaN  2.700000   2.70000  2.700000    3.0   
2022-04-25           4.0  39.69  2.481656   2.80000  2.161100   38.0   
2022-04-26           3.0  26.50  2.410868   7.00000  1.750000   77.0   
2022-04-27           4.0  31.09  2.446802   6.90000  1.815455   63.0   
2022-04-28           1.0  15.90  2.212244   4.62875  0.005000   43.0   

                                      
           unique_buyers      volume  
time                                  
2022-04-24           3.0    8.100000  
2022-04-25          28.0   94.302948  
2022-04-26          62.0  185.636842  
2022-04-27          56.0  154.148555  
2022-04-28          39.0   95.126500  

[5 rows x 36 columns]

## More preprocessing
* In order to do anything very meaningful with the data it is helpful to rename the columns
* We will rename the columns with the prefix "key_" added to each category

In [19]:
cols = ["avg_price", "max_price", "min_price", "trades", "unique_buyers", "volume"]
new_cols = []
for key in contract_ids.keys():
    for c in cols:
        new_cols.append(f"{key}_{c}")
        
new_cols

['bape_avg_price',
 'bape_max_price',
 'bape_min_price',
 'bape_trades',
 'bape_unique_buyers',
 'bape_volume',
 'punks_avg_price',
 'punks_max_price',
 'punks_min_price',
 'punks_trades',
 'punks_unique_buyers',
 'punks_volume',
 'clonex_avg_price',
 'clonex_max_price',
 'clonex_min_price',
 'clonex_trades',
 'clonex_unique_buyers',
 'clonex_volume',
 'doodles_avg_price',
 'doodles_max_price',
 'doodles_min_price',
 'doodles_trades',
 'doodles_unique_buyers',
 'doodles_volume',
 'neotokyo_avg_price',
 'neotokyo_max_price',
 'neotokyo_min_price',
 'neotokyo_trades',
 'neotokyo_unique_buyers',
 'neotokyo_volume',
 'mfers_avg_price',
 'mfers_max_price',
 'mfers_min_price',
 'mfers_trades',
 'mfers_unique_buyers',
 'mfers_volume']

In [20]:
"""
I create a new object of the sum_df with the new columns applied to it. 
I want to leave sum_df the way it is because I will use it for the Monte Carlo simulation later.
"""

'\nI create a new object of the sum_df with the new columns applied to it. \nI want to leave sum_df the way it is because I will use it for the Monte Carlo simulation later.\n'

In [21]:
collection_df = sum_df.copy()
collection_df.columns = new_cols

In [22]:
collection_df.head()

,bape_avg_price,bape_max_price,bape_min_price,bape_trades,bape_unique_buyers,bape_volume,punks_avg_price,punks_max_price,punks_min_price,punks_trades,...,neotokyo_min_price,neotokyo_trades,neotokyo_unique_buyers,neotokyo_volume,mfers_avg_price,mfers_max_price,mfers_min_price,mfers_trades,mfers_unique_buyers,mfers_volume
time,,,,,,,,,,,,,,,,,,,,,
2022-04-24,106.998594,150.000,14.720000,27.0,13.0,2888.962050,70.268571,109.95,59.950000,7.0,...,NaN,NaN,NaN,NaN,2.700000,2.70000,2.700000,3.0,3.0,8.100000
2022-04-25,95.408970,187.005,0.005000,98.0,37.0,9350.079083,66.444761,124.99,24.240945,13.0,...,8.70,4.0,4.0,39.69,2.481656,2.80000,2.161100,38.0,28.0,94.302948
2022-04-26,99.093631,186.000,0.005000,39.0,25.0,3864.651618,81.953636,225.00,61.300000,11.0,...,7.00,3.0,3.0,26.50,2.410868,7.00000,1.750000,77.0,62.0,185.636842
2022-04-27,93.857465,190.000,1.073615,43.0,19.0,4035.871030,76.911428,129.99,57.000000,7.0,...,6.99,4.0,4.0,31.09,2.446802,6.90000,1.815455,63.0,56.0,154.148555
2022-04-28,130.485907,462.000,3.005000,59.0,31.0,7698.668550,68.149166,90.00,61.000000,12.0,...,7.90,2.0,1.0,15.90,2.212244,4.62875,0.005000,43.0,39.0,95.126500


The following is the rolling 30 days standard deviation for each of the collections average price normalized by the average price.

In [23]:
rolling_30_std = collection_df[["bape_avg_price", "clonex_avg_price", "punks_avg_price", "neotokyo_avg_price", "doodles_avg_price", "mfers_avg_price"]].rolling(window=30).std() / collection_df[["bape_avg_price", "clonex_avg_price", "punks_avg_price", "neotokyo_avg_price", "doodles_avg_price", "mfers_avg_price"]] 
rolling_30_std.describe()

,bape_avg_price,clonex_avg_price,punks_avg_price,neotokyo_avg_price,doodles_avg_price,mfers_avg_price
count,62.000000,62.000000,62.000000,0.0,62.000000,62.000000
mean,0.270283,0.221389,0.294910,NaN,0.427794,0.402860
std,0.114425,0.057304,0.104147,NaN,0.273231,0.292351
min,0.149647,0.110630,0.145040,NaN,0.145649,0.117719
25%,0.187568,0.176511,0.219412,NaN,0.227311,0.205997
50%,0.230242,0.219827,0.275467,NaN,0.316273,0.318802
75%,0.316593,0.261892,0.360504,NaN,0.551175,0.442419
max,0.701366,0.362402,0.704978,NaN,1.102739,1.184052


We see that the doodles and mfers have the highest normalized standard deviations and clonex has the lowest. If we were evaluating a loan based solely on std we would apply the greatest collateral discount factor to mfers and doodles 

# A Monte Carlo Projection for our selected collections

### This projection takes the previous 90 days of data and predicts the next 30 days of returns if we held a basket of these NFTs

In [24]:
from MCForecastTools import MCSimulation

In [33]:
# simulation set to iterate 100 times over the next 30 trading days
# we leave the default weights which will be 1/6 per collection

# in the MCForecastTools.py file the 'close' column was changed to 'avg_price' to fit our data
sim = MCSimulation(sum_df, num_simulation=100, num_trading_days=30)

In [34]:
sim.portfolio_data.head()

bape                                                         \
             avg_price max_price  min_price trades unique_buyers       volume   
time                                                                            
2022-04-24  106.998594   150.000  14.720000   27.0          13.0  2888.962050   
2022-04-25   95.408970   187.005   0.005000   98.0          37.0  9350.079083   
2022-04-26   99.093631   186.000   0.005000   39.0          25.0  3864.651618   
2022-04-27   93.857465   190.000   1.073615   43.0          19.0  4035.871030   
2022-04-28  130.485907   462.000   3.005000   59.0          31.0  7698.668550   

                             punks                       ...      neotokyo  \
           daily_return  avg_price max_price  min_price  ... unique_buyers   
time                                                     ...                 
2022-04-24          NaN  70.268571    109.95  59.950000  ...           NaN   
2022-04-25    -0.108316  66.444761    124.99  24.240945  ...           4.0   
2022-04-26     0.038620  81.953636    225.00  61.300000  ...           3.0   
2022-04-27    -0.052841  76.911428    129.99  57.000000  ...           4.0   
2022-04-28     0.390256  68.149166     90.00  61.000000  ...           1.0   

                                   mfers                             \
           volume daily_return avg_price max_price min_price trades   
time                                                                  
2022-04-24    NaN          NaN  2.700000   2.70000  2.700000    3.0   
2022-04-25  39.69          NaN  2.481656   2.80000  2.161100   38.0   
2022-04-26  26.50    -0.109767  2.410868   7.00000  1.750000   77.0   
2022-04-27  31.09    -0.120094  2.446802   6.90000  1.815455   63.0   
2022-04-28  15.90     0.022837  2.212244   4.62875  0.005000   43.0   

                                                   
           unique_buyers      volume daily_return  
time                                               
2022-04-24           3.0    8.100000          NaN  
2022-04-25          28.0   94.302948    -0.080868  
2022-04-26          62.0  185.636842    -0.028525  
2022-04-27          56.0  154.148555     0.014905  
2022-04-28          39.0   95.126500    -0.095863  

[5 rows x 42 columns]

In [27]:
display(f"bape: {sim.portfolio_data['bape']['daily_return'].mean()}")
display(f"punks: {sim.portfolio_data['punks']['daily_return'].mean()}")
display(f"clonex: {sim.portfolio_data['clonex']['daily_return'].mean()}")
display(f"doodles: {sim.portfolio_data['doodles']['daily_return'].mean()}")
display(f"neotokyo: {sim.portfolio_data['neotokyo']['daily_return'].mean()}")
display(f"mfers: {sim.portfolio_data['mfers']['daily_return'].mean()}")

'bape: 0.06690547084531029'

'punks: 0.06143570518955664'

'clonex: 0.01335312028719893'

'doodles: 0.0997605877430711'

'neotokyo: 0.18298310075282617'

'mfers: 0.06097347838567557'

We see that these collections have all performed strongly over the last ninety days, each with a positive average daily return

In [35]:
cum_return = sim.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


In [37]:
cum_return.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [38]:
cum_return.describe()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,...,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [39]:
cum_return.mean().mean()

1.0

Based on the forecasted returns for this basket of NFTs it would be a good selection candidate for collateralization

# Beta Analysis For NFT versus Basket

* What we want to do here is find the relative risk for each asset in the basket versus the basket as a whole.
* For instance, we will compare Crypto Punks, etc. to the 6 NFT collection that we selected using Beta.


In [57]:
collection_df = collection_df.drop("bape_pct_chg", axis=1)

In [63]:
def find_pct_change(df, contract_ids):
    coll_names = []
    counter = 0
    for k in contract_ids.keys():
        coll_names.append(k)
    for col in df.columns:
        if "avg_price" in col:
            df[f"{coll_names[counter]}_pct_chg"] = df[col].pct_change()
            counter += 1
    return df
            

In [64]:
find_pct_change(collection_df, contract_ids)

,bape_avg_price,bape_max_price,bape_min_price,bape_trades,bape_unique_buyers,bape_volume,punks_avg_price,punks_max_price,punks_min_price,punks_trades,...,mfers_min_price,mfers_trades,mfers_unique_buyers,mfers_volume,bape_pct_chg,punks_pct_chg,clonex_pct_chg,doodles_pct_chg,neotokyo_pct_chg,mfers_pct_chg
time,,,,,,,,,,,,,,,,,,,,,
2022-04-24,106.998594,150.000,14.720000,27.0,13.0,2888.962050,70.268571,109.95,59.950000,7.0,...,2.700000,3.0,3.0,8.100000,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-25,95.408970,187.005,0.005000,98.0,37.0,9350.079083,66.444761,124.99,24.240945,13.0,...,2.161100,38.0,28.0,94.302948,-0.108316,-0.054417,-0.180087,-0.056975,NaN,-0.080868
2022-04-26,99.093631,186.000,0.005000,39.0,25.0,3864.651618,81.953636,225.00,61.300000,11.0,...,1.750000,77.0,62.0,185.636842,0.038620,0.233410,0.310340,0.371297,-0.109767,-0.028525
2022-04-27,93.857465,190.000,1.073615,43.0,19.0,4035.871030,76.911428,129.99,57.000000,7.0,...,1.815455,63.0,56.0,154.148555,-0.052841,-0.061525,-0.160842,-0.262632,-0.120094,0.014905
2022-04-28,130.485907,462.000,3.005000,59.0,31.0,7698.668550,68.149166,90.00,61.000000,12.0,...,0.005000,43.0,39.0,95.126500,0.390256,-0.113927,-0.039875,0.079584,0.022837,-0.095863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-19,92.779680,109.000,84.517500,5.0,4.0,463.898400,96.825000,169.00,77.000000,6.0,...,1.406000,20.0,19.0,38.843450,0.204465,0.086853,0.058956,-0.374076,1.343957,-0.149943
2022-07-20,38.500000,38.500,38.500000,1.0,1.0,38.500000,90.460000,102.50,76.000000,9.0,...,1.328100,11.0,8.0,15.372350,-0.585038,-0.065737,0.004970,0.050221,-0.317746,-0.280452
2022-07-21,75.936816,92.150,17.000000,6.0,6.0,455.620900,69.642533,94.00,20.159963,7.0,...,0.023046,16.0,10.0,21.268354,0.972385,-0.230129,-0.192636,-0.035533,0.183043,-0.048812


In [90]:
def basket_pct_chg(df, contract_ids):
    coll_names = []
    pct_chg_lst = []
    for k in contract_ids.keys():
        coll_names.append(k)
    for col in df.columns:
        if "pct_chg" in col:
            pct_chg_lst.append(col)
    basket_df = df[pct_chg_lst]
    return basket_df.dropna()
            
        

In [91]:
basket_df = basket_pct_chg(collection_df, contract_ids)
basket_df["basket_pct_chg"] = basket_df[basket_df.columns].mean(axis=1)

basket_df

,bape_pct_chg,punks_pct_chg,clonex_pct_chg,doodles_pct_chg,neotokyo_pct_chg,mfers_pct_chg,basket_pct_chg
time,,,,,,,
2022-04-26,0.038620,0.233410,0.310340,0.371297,-0.109767,-0.028525,0.135896
2022-04-27,-0.052841,-0.061525,-0.160842,-0.262632,-0.120094,0.014905,-0.107172
2022-04-28,0.390256,-0.113927,-0.039875,0.079584,0.022837,-0.095863,0.040502
2022-04-29,0.032359,-0.120895,-0.062435,-0.088519,0.056604,-0.022859,-0.034291
2022-04-30,0.011024,0.054875,-0.130123,-0.197778,0.116468,-0.014092,-0.026604
...,...,...,...,...,...,...,...
2022-07-19,0.204465,0.086853,0.058956,-0.374076,1.343957,-0.149943,0.195035
2022-07-20,-0.585038,-0.065737,0.004970,0.050221,-0.317746,-0.280452,-0.198964
2022-07-21,0.972385,-0.230129,-0.192636,-0.035533,0.183043,-0.048812,0.108053


In [105]:

bape_beta = basket_df["bape_pct_chg"].cov(basket_df["basket_pct_chg"]) / basket_df["basket_pct_chg"].var()
punks_beta = basket_df["punks_pct_chg"].cov(basket_df["basket_pct_chg"]) / basket_df["basket_pct_chg"].var()
neo_beta = basket_df["neotokyo_pct_chg"].cov(basket_df["basket_pct_chg"]) / basket_df["basket_pct_chg"].var()
clonex_beta = basket_df["clonex_pct_chg"].cov(basket_df["basket_pct_chg"]) / basket_df["basket_pct_chg"].var()
doodles_beta = basket_df["doodles_pct_chg"].cov(basket_df["basket_pct_chg"]) / basket_df["basket_pct_chg"].var()
mfers_beta = basket_df["mfers_pct_chg"].cov(basket_df["basket_pct_chg"]) / basket_df["basket_pct_chg"].var()


In [131]:
betas_list = [bape_beta, punks_beta, neo_beta, clonex_beta, doodles_beta, mfers_beta]
betas = pd.DataFrame([bape_beta, punks_beta, neo_beta, clonex_beta, doodles_beta, mfers_beta], index=contract_ids.keys())
betas.hvplot.bar()

:Bars   [index]   (0)

## Conclusions

From the beta analysis data, we would be more inclined to use the collections with the lowest beta values as collateral because they indicate greater security against the market. In this case, Crypto Punks and Doodles serve as the best candidates for collateralization and borrowers would be rewarded with a potentially lower collateral discount factor.

# Epilogue
## Foreshadowing a Collateral Discount Curve Based on Beta Values

Let's say that we want to appraise how much of a collateral discount factor to apply to a token/collection based on its beta value. In terms of Beta values, a higher one should correspond to a higher discount factor applied to the asset. If an asset is more risky, we can only safely provide a smaller loan. For collateral assets the holy grail is high stability and is even better if it is highly stable appreciation (ie. real estate). If a borrower defaults on a loan it is reassuring to know that the asset used as collateral has either the same value or a higher value than when we received it.


In [133]:
def discount_factor(betas):
    discount_factors = []
    for beta in betas:
        disc_factor = 1 - (1/(beta + 1.5)) + .1697
        discount_factors.append(disc_factor)
    return discount_factors

In [136]:
discount_factors_list = discount_factor(betas_list)

In [139]:
discount_factors = pd.DataFrame(discount_factors_list, index=contract_ids.keys())
discount_factors.hvplot.bar()

:Bars   [index]   (0)

In [140]:
discount_factors

,0
bape,0.698132
punks,0.576658
clonex,0.945944
doodles,0.603893
neotokyo,0.795577
mfers,0.731822


## Let's Say
I'm a user with a Doodles NFT that is worth 15eth and I am looking for a loan. Based on the discount curve what kind of a loan could I expect to receive for my NFT?

In [144]:
def find_loan_value(collection: str, value: float, discount_factors):
    loan_value = value - value * discount_factors[0][collection]
    return loan_value

In [147]:
loan_value = find_loan_value("doodles", 15, discount_factors)

In [150]:
print(f"The loan calculator has determined that you are eligible to receive {loan_value: .2f}eth for your NFT")

The loan calculator has determined that you are eligible to receive  5.94eth for your NFT
